In [30]:
!ls

 Attribution_Patching_Demo.ipynb
 Dockerfile
 LICENSE
 README.md
 __pycache__
 adjective_token_lengths.txt
 ccs.py
 ccs_act_patching.py
 ccs_circuit_analysis.py
 ccs_circuit_attribution.py
 ccs_circuit_path_patching.py
 circuit-observations.md
 circuit.md
 circuit_analysis_classification_prompt_experimentation_pythia2_8b.ipynb
 circuit_analysis_contrastive_sentiment_gpt2_small.py
 circuit_analysis_restaurant_review_classification_pythia1_4b.ipynb
'circuit_analysis_sentiment continuation_pythia1_4b.py'
 circuit_analysis_sentiment_classification_pythia1_4b.ipynb
 circuit_analysis_sentiment_classification_pythia1_4b.py
 circuit_analysis_sentiment_continuation_pythia1_4b.ipynb
 circuit_analysis_sentiment_contradiction_pythia1_4b.ipynb
'circuit_analysis_simple single sentiment_gpt2_small.py'
 circuit_analysis_simple_sentiment_gpt2_small.py
'circuit_analysis_task comparison_pythia1_4b.ipynb'
 circuit_for_mood_binding_pythia2_8b.ipynb
 circuit_for_mood_inference_pythia2_8b.ipynb
 circuit_for_

In [2]:
%cd eliciting-latent-sentiment

/notebooks/eliciting-latent-sentiment


In [3]:
!pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
!pip install transformer_lens
!pip install jaxtyping==0.2.13
!pip install einops
!pip install protobuf==3.20.*
!pip install plotly
!pip install torchtyping
!pip install git+https://github.com/neelnanda-io/neel-plotly.git
# !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
# %pip install git+https://github.com/neelnanda-io/PySvelte.git
# %pip install typeguard==2.13.3

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-cb5c1m6a
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-cb5c1m6a
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit 5afe6fed827592dd525490b81e213bc3e2241a4a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 21.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 8.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 13.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 114.9 MB/s eta 0:00:0000

In [4]:
from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

In [5]:
import einops
from functools import partial
import torch
from torch import Tensor
from torch.utils.data import DataLoader
from datasets import load_dataset
from jaxtyping import Float, Int, Bool
from typing import Dict, Iterable, List, Tuple, Union
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_dataset, tokenize_and_concatenate, get_act_name, test_prompt
from transformer_lens.hook_points import HookPoint
from tqdm.notebook import tqdm
import pandas as pd
from circuitsvis.activations import text_neuron_activations
from utils.store import load_array, save_html, save_array, is_file, get_model_name, clean_label, save_text

In [6]:
torch.set_grad_enabled(False)
device = "cuda"
MODEL_NAME = "EleutherAI/pythia-2.8b"
model = HookedTransformer.from_pretrained(
    MODEL_NAME,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=False,
    device=device,
)
model.name = MODEL_NAME

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-2.8b into HookedTransformer


In [7]:
BATCH_SIZE = 24
owt_data = load_dataset("stas/openwebtext-10k", split="train")
dataset = tokenize_and_concatenate(owt_data, model.tokenizer)
data_loader = DataLoader(
    dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True
)

Repo card metadata block was not found. Setting CardData to empty.


/usr/local/lib/python3.9/dist-packages/datasets/download/download_manager.py:527: FutureWarning: 'num_proc' was deprecated in version 2.6.2 and will be removed in 3.0.0. Pass `DownloadConfig(num_proc=<num_proc>)` to the initializer instead.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
def find_positions(tensor, token_ids=[11, 13]):
    positions = []
    for batch_item in tensor:
        token_positions = {token_id: [] for token_id in token_ids}
        for position, token in enumerate(batch_item):
            if token.item() in token_ids:
                token_positions[token.item()].append(position)
        positions.append([token_positions[token_id] for token_id in token_ids])
    return positions

In [9]:
def zero_attention_pos_hook(
    pattern: Float[Tensor, "batch head seq_Q seq_K"], hook: HookPoint,
    pos_by_batch: List[List[int]], layer: int = 0, head_idx: int = 0,
) -> Float[Tensor, "batch head seq_Q seq_K"]:
    """Zero-ablates an attention pattern tensor at a particular position"""
    assert 'pattern' in hook.name

    batch_size = pattern.shape[0]
    assert len(pos_by_batch) == batch_size

    for i in range(batch_size):
        for p in pos_by_batch[i]:
            pattern[i, head_idx, p, p] = 0
            
    return pattern



In [10]:
def names_filter(name: str):
    """Filter for the names of the activations we want to keep to study the resid stream."""
    return name.endswith('resid_post') or name == get_act_name('resid_pre', 0)

def get_layerwise_token_mean_activations(model: HookedTransformer, data_loader: DataLoader, token_id: int = 13) -> Float[Tensor, "layer d_model"]:
    """Get the mean value of a token across layers"""
    num_layers = model.cfg.n_layers
    d_model = model.cfg.d_model
    
    activation_sums = torch.stack([torch.zeros(d_model) for _ in range(num_layers)]).to(device)
    comma_counts = [0] * num_layers

    print(activation_sums.shape)

    token_mean_values = torch.zeros((num_layers, d_model))
    for _, batch_value in tqdm(enumerate(data_loader), total=100):
        
        batch_tokens = batch_value['tokens'].to(device)

        # get positions of all 11 and 13 token ids in batch
        punct_pos = find_positions(batch_tokens, token_ids=[13])

        _, cache = model.run_with_cache(
            batch_tokens, 
            names_filter=names_filter
        )

        
        for i in range(batch_tokens.shape[0]):
            for p in punct_pos[i][0]:
                for layer in range(num_layers):
                    activation_sums[layer] += cache[f"blocks.{layer}.hook_resid_post"][i, p, :]
                    comma_counts[layer] += 1

    for layer in range(num_layers):
        token_mean_values[layer] = activation_sums[layer] / comma_counts[layer]

    return token_mean_values

In [10]:
comma_mean_values = get_layerwise_token_mean_activations(model, data_loader, token_id=13)

torch.Size([32, 2560])


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
comma_mean_values.shape

torch.Size([32, 2560])

In [15]:
save_array(comma_mean_values, 'comma_mean_values.npy', model)

'data/pythia-2.8b/comma_mean_values.npy'

In [11]:
# load the file
comma_mean_values = torch.from_numpy(load_array('comma_mean_values.npy', model)).to(device)

In [53]:
def plot_neuroscope(
    tokens: Int[Tensor, "batch pos"], centred: bool = False, activations: Float[Tensor, "pos layer 1"] = None,
    verbose=False,
):
    
    str_tokens = model.to_str_tokens(tokens, prepend_bos=False)

    if verbose:
        print(f"Tokens shape: {tokens.shape}")
  
    if centred:
        if verbose:
            print("Centering activations")
        layer_means = einops.reduce(activations, "pos layer 1 -> 1 layer 1", reduction="mean")
        layer_means = einops.repeat(layer_means, "1 layer 1 -> pos layer 1", pos=activations.shape[0])
        activations -= layer_means
    elif verbose:
        print("Activations already centered")
    assert (
        activations.ndim == 3
    ), f"activations must be of shape [tokens x layers x neurons], found {activations.shape}"
    assert len(str_tokens) == activations.shape[0], (
        f"tokens and activations must have the same length, found tokens={len(str_tokens)} and acts={activations.shape[0]}, "
        f"tokens={str_tokens}, "
        f"activations={activations.shape}"

    )
    return text_neuron_activations(
        tokens=str_tokens, 
        activations=activations,
        first_dimension_name="Layer (resid_pre)",
        second_dimension_name="Model",
        second_dimension_labels=["pythia-2.8b"],
    )

In [59]:
from utils.ablation import ablate_resid_with_precalc_mean

heads_to_ablate = [(layer, head) for layer in range(model.cfg.n_layers) for head in range(model.cfg.n_heads)]

def compute_mean_ablation_modified_loss(model: HookedTransformer, data_loader: DataLoader, cached_means) -> float:
    total_loss = 0
    loss_list = []
    it = 0
    for _, batch_value in tqdm(enumerate(data_loader), total=len(data_loader)):
        batch_tokens = batch_value['tokens'].to(device)

        # get positions of all 11 and 13 token ids in batch
        punct_pos = find_positions(batch_tokens, token_ids=[13])

        # get the loss for each token in the batch
        initial_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)
        
        # add hooks for the activations of the 11 and 13 tokens
        for layer, head in heads_to_ablate:
            mean_ablate_comma = partial(ablate_resid_with_precalc_mean, cached_means=cached_means, pos_by_batch=punct_pos, layer=layer)
            model.blocks[layer].hook_resid_post.add_hook(mean_ablate_comma)

        # get the loss for each token when run with hooks
        hooked_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)

        # compute the percent difference between the two losses
        loss_diff = (hooked_loss - initial_loss) / initial_loss
        loss_list.append(loss_diff)
        it += 1
        if it == 10:
            break

    model.reset_hooks()
    return loss_list

# def compute_modified_loss(model: HookedTransformer, data_loader: DataLoader) -> float:
#     total_loss = 0
#     loss_list = []
#     for _, batch_value in tqdm(enumerate(data_loader), total=len(data_loader)):
#         batch_tokens = batch_value['tokens'].to(device)

#         # get positions of all 11 and 13 token ids in batch
#         punct_pos = find_positions(batch_tokens, token_ids=[13])

#         # get the loss for each token in the batch
#         initial_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)
        
#         # add hooks for the activations of the 11 and 13 tokens
#         for layer, head in heads_to_ablate:
#             ablate_punct = partial(zero_attention_pos_hook, pos_by_batch=punct_pos, layer=layer, head_idx=head)
#             model.blocks[layer].attn.hook_pattern.add_hook(ablate_punct)

#         # get the loss for each token when run with hooks
#         hooked_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)

#         # compute the percent difference between the two losses
#         loss_diff = (hooked_loss - initial_loss) / initial_loss

#         loss_list.append(loss_diff)

#     model.reset_hooks()
#     return loss_list, batch_tokens

In [60]:
model.to_tokens(",")

tensor([[ 0, 13]], device='cuda:0')

In [62]:
load_dataset?

Signature:
load_dataset(
    path: str,
    name: Optional[str] = None,
    data_dir: Optional[str] = None,
    data_files: Union[str, Sequence[str], Mapping[str, Union[str, Sequence[str]]], NoneType] = None,
    split: Union[str, datasets.splits.Split, NoneType] = None,
    cache_dir: Optional[str] = None,
    features: Optional[datasets.features.features.Features] = None,
    download_config: Optional[datasets.download.download_config.DownloadConfig] = None,
    download_mode: Union[datasets.download.download_manager.DownloadMode, str, NoneType] = None,
    verification_mode: Union[datasets.utils.info_utils.VerificationMode, str, NoneType] = None,
    ignore_verifications='deprecated',
    keep_in_memory: Optional[bool] = None,
    save_infos: bool = False,
    revision: Union[str, datasets.utils.version.Version, NoneType] = None,
    token: Union[bool, str, NoneType] = None,
    use_auth_token='deprecated',
    task='deprecated',
    streaming: bool = False,
    num_proc: Optional[i

In [61]:
loss_change_by_token = compute_mean_ablation_modified_loss(model, data_loader, comma_mean_values)
#loss_change_by_token, batch_tokens = compute_modified_loss(model, data_loader)
#loss_change_by_token[11]

  0%|          | 0/456 [00:00<?, ?it/s]

In [63]:
from torch.utils.data import DataLoader, Subset

# Assuming data_loader is already defined as in your code
# Create a subset of the dataset containing only the first ten items
subset_dataset = Subset(dataset, indices=range(10))

# Create a new dataloader from the subset
# You can adjust the batch size and other parameters as needed
subset_data_loader = DataLoader(
    subset_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True
)

# Now, subset_data_loader contains the first ten items from the original dataloader

In [43]:
len(loss_change_by_token)

10

In [66]:
for i in range(len(loss_change_by_token)):
    # add one column of zeros to the loss change tensor
    loss_change_by_token[i] = torch.cat([torch.zeros(loss_change_by_token[i].shape[0], 1).to(device), loss_change_by_token[i]], dim=1)

In [67]:
loss_change_by_token = torch.stack(loss_change_by_token).cpu().numpy()

In [68]:
save_array(loss_change_by_token, 'loss_change_by_token.npy', model)

'data/pythia-2.8b/loss_change_by_token.npy'

In [53]:
model.reset_hooks()

In [70]:
load_array?

Signature:
load_array(
    label: str,
    model: Union[transformer_lens.HookedTransformer.HookedTransformer, str],
) -> numpy.ndarray
Docstring: <no docstring>
File:      /notebooks/eliciting-latent-sentiment/utils/store.py
Type:      function


In [72]:
from utils.neuroscope import plot_topk
loss_change_by_token = torch.from_numpy(load_array('loss_change_by_token.npy', model))
plot_topk(activations=loss_change_by_token, dataloader=subset_data_loader, model=model, k=5)

Top 5 most positive examples:


TypeError: extract_text_window() missing 2 required positional arguments: 'dataloader' and 'model'

In [35]:
plot_neuroscope(batch_tokens[11], activations=loss_change_by_token[11].unsqueeze(1).unsqueeze(2))

In [77]:
loss_change_by_token[11].max()

tensor(607.2508, device='cuda:0')

In [62]:
loss_change_by_token[11].unsqueeze(1).unsqueeze(2).shape

torch.Size([1024, 1, 1])

In [67]:
batch_tokens[11].unsqueeze(0)

tensor([[    0,   604, 40537,  ...,  4163,   846,  1534]], device='cuda:0')